# Rivulet (Weather): National Weather Service (NWS) API
_by Danny Zheng_

## Purpose of this Notebook

This notebook was developed as part of NSF Grant 2445609 to support accessing and processing public data for middle and high school classroom activities. It's written to be relatively accessible to beginners, but if you have not interacted with computational notebooks or Python before, you may find navigating this tool difficult. (Check out the Show Your Work project for a gentle introduction to computational notebooks for educators!)

Our project is focused on supporting data analysis and mechanistic reasoning in science education. In other words, we want students to learn how data provides information about how scientific mechanisms work and how understanding scientific mechanisms can help them to explain and interpret patterns in data. This builds on a long history of research on complex systems and agent-based modeling, and more closely connects that work to current expansions of data analysis across subjects.

This tool focuses on connecting to the National Weather Service (NWS) API to access public weather data. The goal is to gather meteorological data (such as precipitation, air temperature, etc.) that can be used to explain and interpret patterns in water quality data. This supports the project's focus on helping students learn how data provides information about *how scientific mechanisms work*.

## Part I: Setup and Location

The NWS API is public and does not require a secret key. We will define our imports and location in one place to streamline the process. The NWS API uses a latitude/longitude point to find weather data.

In [ ]:
# Install necessary libraries for the NWS API and mapping
!pip install requests folium

# Import libraries
import requests
import pandas as pd
import folium #Used to map

# The NWS API requires a User-Agent for identification, NOT a secret key.
# IMPORTANT: Replace the email below with your actual email.
USER_AGENT = "(Rivulet Project, dazzy0130@gmail.com)"

# EDIT HERE: Define the target latitude and longitude for your region.
lat = 37.6213  #  San Francisco International Airport (SFO)
lon = -122.3896 #  San Francisco International Airport (SFO)

## Part II: Specifying a Location and Visualizing

Since the NWS API queries a single point (latitude and longitude), we will map this point to confirm the exact location being requested.

In [ ]:
# Map the location
map_center = [lat, lon]

# Create a Folium map object
m = folium.Map(location=map_center, zoom_start=12)

# Add a marker to the map for the exact point
folium.Marker(
    location=[lat, lon],
    tooltip="Target NWS Location"
).add_to(m)

# Display the map
m

## Part III: Establishing the Connection

This connection request is the foundational step. It retrieves the necessary metadata (the weather **grid ID** and **forecast URL**) that will be used for all subsequent weather data requests.

In [ ]:
# This API endpoint finds the correct weather grid for a given coordinate
api_endpoint = f"https://api.weather.gov/points/{lat},{lon}"

# Set up the headers with our User-Agent (defined in Cell 6)
# Note: The NWS API requires this User-Agent header.
headers = {
    'User-Agent': USER_AGENT
}

# 1. Make the request to the NWS API
response = requests.get(api_endpoint, headers=headers)

# 2. Convert the successful JSON response into a Python dictionary
api_data = response.json()

# The final output of the cell is the data dictionary, showing the key properties.
# Future steps will extract the URLs (like 'forecast') from this dictionary.
api_data['properties']

## Part IV: Wind Vector Transformation for Mechanistic Reasoning

We need historical wind data that is structured for easy analysis. This requires two steps: first, finding the nearest observation station with historical records, and second, fetching that data and converting the raw wind speed and direction into u (East-West) and v (North-South) vector components. This transformation is necessary to analyze wind changes over time.

In [ ]:
# Install and import numpy, which is necessary for vector math
!pip install -q numpy
import numpy as np
from datetime import datetime, timedelta

In [ ]:
stations_url = api_data['properties']['observationStations']

# Request the list of stations. Note: headers are already defined in Cell 6.
stations_response = requests.get(stations_url, headers=headers)
stations_response.raise_for_status()

# Extract data and find a reliable station (e.g., index 1 often reports complete wind data).
stations_data = stations_response.json()
nearest_station_url = stations_data['features'][1]['properties']['@id']

# The output of the cell is the station URL, which is used in the next step.
nearest_station_url

In [ ]:
# Define the required time format for the NWS API
TIME_FORMAT = "%Y-%m-%dT%H:%M:%SZ"

# EDIT HERE: Define the time period (e.g., past 7 days)
end_time = datetime.now().strftime(TIME_FORMAT)
start_time = (datetime.now() - timedelta(days=7)).strftime(TIME_FORMAT)

observation_endpoint = f"{nearest_station_url}/observations?start={start_time}&end={end_time}"

# Request the actual observation data
observation_response = requests.get(observation_endpoint, headers=headers)
observation_response.raise_for_status()

observations = observation_response.json()

# --- Data Transformation and Vector Calculation ---
clean_observations = []
for obs in observations['features']:
    props = obs['properties']

    # Only process records that contain both wind speed and direction
    if props['windSpeed']['value'] is not None and props['windDirection']['value'] is not None:

        speed = props['windSpeed']['value']
        direction = props['windDirection']['value']

        # Convert direction to radians for vector math
        direction_rad = np.deg2rad(direction)

        # Calculate u (East-West) and v (North-South) components
        u_component = -speed * np.sin(direction_rad)
        v_component = -speed * np.cos(direction_rad)

        clean_observations.append({
            'timestamp': props['timestamp'],
            'wind_speed_m_s': speed,
            'wind_direction_deg': direction,
            'u_component_m_s': u_component,
            'v_component_m_s': v_component
        })

# Convert the results to a DataFrame
df_wind_data = pd.DataFrame(clean_observations)

# Display the final, transformed DataFrame
df_wind_data[['timestamp', 'wind_speed_m_s', 'u_component_m_s', 'v_component_m_s']].head()